In [1]:
import os
import numpy as np
import pandas as pd

# Завантаження файлу з локального комп'ютеру

In [3]:
DATA_DIR = "/Users/sonyakoldun/downloads"
target_name = "raw_customer_visists_filtered.csv"

target_file = os.path.join(DATA_DIR, target_name)
data = pd.read_csv(target_file)

# Перетворення дати та візуальна перевірка колонок

In [5]:
data["date"] = pd.to_datetime(data["date"])
data.head()

,tab_id,user_id,date,venue_chain_id,venue_id
0,000a45b6-ff23-4d42-95b7-732ee8887d2e,6a0c9cd3-edd7-08dd-ec10-fcf2def42345,2023-12-01,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
1,000c7df0-fed1-46e0-bc05-cf2b2b193c9f,CASH,2023-11-17,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
2,000d2405-9ee8-4cfb-ae2a-45200ef2c8f7,cc8f8d2e-4e62-a29f-ab53-10e268475a90,2023-12-21,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
3,000edf49-4595-4afd-8ae3-990e60ae6c57,cf8b1a82-f0f6-07bd-ad8b-2461ca248849,2023-11-14,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38
4,001631be-c413-4aa0-b388-51170ce932bd,CASH,2023-12-23,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,38


In [6]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 817207 entries, 0 to 817206
Data columns (total 5 columns):
 #   Column          Non-Null Count   Dtype         
---  ------          --------------   -----         
 0   tab_id          817207 non-null  object        
 1   user_id         817207 non-null  object        
 2   date            817207 non-null  datetime64[ns]
 3   venue_chain_id  817207 non-null  object        
 4   venue_id        817207 non-null  int64         
dtypes: datetime64[ns](1), int64(1), object(3)
memory usage: 31.2+ MB


### кожен user_id == 'CASH' сприйматиметься як новий клієнт, бо відтрекати чи цей клієнт вже був - неможливо

In [8]:
card_data = data[data["user_id"] != "CASH"].copy()
cash_data = data[data["user_id"] == "CASH"].copy()

card_data.head(), cash_data.head()

(                                 tab_id                               user_id  \
 0  000a45b6-ff23-4d42-95b7-732ee8887d2e  6a0c9cd3-edd7-08dd-ec10-fcf2def42345   
 2  000d2405-9ee8-4cfb-ae2a-45200ef2c8f7  cc8f8d2e-4e62-a29f-ab53-10e268475a90   
 3  000edf49-4595-4afd-8ae3-990e60ae6c57  cf8b1a82-f0f6-07bd-ad8b-2461ca248849   
 5  0018dea5-14a1-4814-86bb-4ecb08fc90ce  1eaab6fa-9b24-69fc-199e-8c5ed1eaa38d   
 6  001ae2a7-c346-4aee-bfa5-994d39505b2f  62a29f77-3a84-7d4d-f14b-b8007cc879b3   
 
         date                        venue_chain_id  venue_id  
 0 2023-12-01  14eaeec5-2fd0-621a-49aa-4a8090cabcf7        38  
 2 2023-12-21  14eaeec5-2fd0-621a-49aa-4a8090cabcf7        38  
 3 2023-11-14  14eaeec5-2fd0-621a-49aa-4a8090cabcf7        38  
 5 2023-11-25  14eaeec5-2fd0-621a-49aa-4a8090cabcf7        38  
 6 2024-03-21  14eaeec5-2fd0-621a-49aa-4a8090cabcf7        38  ,
                                   tab_id user_id       date  \
 1   000c7df0-fed1-46e0-bc05-cf2b2b193c9f    CASH 2023-11

вважаю, що кожний tab = окремий візит 
проміжок беру в 365 днів, тобто календарний рік 

In [10]:
def classify_card_visits(card_df: pd.DataFrame) -> pd.DataFrame:
    # сортуємо дані 
    group_cols = ["venue_chain_id", "venue_id", "user_id"]
    card_df = card_df.sort_values(group_cols + ["date"]).copy()

    # дата попереднього візиту цього клієнта 
    card_df["prev_date"] = card_df.groupby(group_cols)["date"].shift(1)

    # чкільки днів пройшло між візитами
    card_df["delta_days"] = (card_df["date"] - card_df["prev_date"]).dt.days

    # новий цикл якщо це перший візит, або якщо між візитами пройшло ≥ 365 днів
    card_df["new_cycle"] = card_df["prev_date"].isna() | (card_df["delta_days"] >= 365)

    # номер циклу для кожного клієнта
    card_df["cycle_id"] = card_df.groupby(group_cols)["new_cycle"].cumsum()

    # номер візиту всередині циклу (1, 2, 3…)
    card_df["visit_order"] = card_df.groupby(group_cols + ["cycle_id"]).cumcount() + 1

    # тип візиту
    card_df["visit_type"] = np.where(
        card_df["visit_order"] == 1, "new",
        np.where(card_df["visit_order"] == 2, "first_return", "repeated_return")
    )

    # дата першого візиту в циклі (для конверсії)
    first_visit_date = card_df.groupby(group_cols + ["cycle_id"])["date"].transform("first")

    # скільки днів між "new" і "first_return"
    card_df["conversion_days"] = np.where(
        card_df["visit_order"] == 2,
        (card_df["date"] - first_visit_date).dt.days,
        np.nan
    )

    return card_df

In [11]:
card_visits = classify_card_visits(card_data)
card_visits.head()

,tab_id,user_id,date,venue_chain_id,venue_id,prev_date,delta_days,new_cycle,cycle_id,visit_order,visit_type,conversion_days
732369,a8ed8dbe-3e13-44e6-879d-48a9486dbf80,00008bcb-88a6-fc9a-a9e3-919154cde457,2022-01-21,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,NaT,NaN,True,1,1,new,NaN
755249,c0a6ef75-b90e-4e98-967b-05654e42935b,00008bcb-88a6-fc9a-a9e3-919154cde457,2022-01-21,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2022-01-21,0.0,False,1,2,first_return,0.0
753124,be7f56c8-26aa-43aa-8d4e-ab15c4ae7c80,0000dd41-e592-abf1-6bc7-3cf28cfb34a5,2023-08-12,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,NaT,NaN,True,1,1,new,NaN
703220,8b2bf8eb-3eed-436a-955f-60bfa9865263,0001010b-7eb3-43dc-3108-494e9f864af9,2021-06-21,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,NaT,NaN,True,1,1,new,NaN
771409,d1253712-122f-4b88-a553-b036e5e858fa,0001010b-7eb3-43dc-3108-494e9f864af9,2021-09-09,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-06-21,80.0,False,1,2,first_return,80.0


### додаю колонку місяців

In [13]:
card_visits["month"] = card_visits["date"].dt.to_period("M").astype(str)
cash_data["month"] = cash_data["date"].dt.to_period("M").astype(str)

In [14]:
# кількість візитів карткових клієнтів за типом візиту
grouped = card_visits.groupby(
    ["venue_chain_id", "venue_id", "month", "visit_type"]
)["user_id"].count()

# типи візитів у колонки
card_counts = grouped.unstack("visit_type", fill_value=0).reset_index()

# кожен user_id == 'CASH' новий клієнт
cash_counts = (
    cash_data
    .groupby(["venue_chain_id", "venue_id", "month"])["tab_id"]
    .count()
    .reset_index(name="new_customers_cash")
)

In [15]:
# рахуємо кількість візитів за типом для карткових
card_counts = (
    card_visits
    .groupby(["venue_chain_id", "venue_id", "month", "visit_type"])
    .size()
    .unstack("visit_type", fill_value=0)
    .reset_index()
)

# переназиваю для зрозумілості бо далі буде + одна колонка
card_counts = card_counts.rename(columns={
    "new": "new_customers_card",
    "first_return": "first_return_customers",
    "repeated_return": "repeated_return_customers"
})


# рахую кількість візитів за типом для готівкових
cash_counts = (
    cash_data
    .groupby(["venue_chain_id", "venue_id", "month"])
    .size()
    .reset_index(name="new_customers_cash")
)


# об’єдную
monthly_venue = (
    pd.merge(
        card_counts,
        cash_counts,
        on=["venue_chain_id", "venue_id", "month"],
        how="outer"
    )
    .fillna(0)
)


# загальна кількість нових клієнтів
monthly_venue["new_customers"] = (
    monthly_venue["new_customers_card"] +
    monthly_venue["new_customers_cash"]
)


monthly_venue.head()

,venue_chain_id,venue_id,month,first_return_customers,new_customers_card,repeated_return_customers,new_customers_cash,new_customers
0,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-06,611,3396,225,747,4143
1,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-07,1014,4472,641,1050,5522
2,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-08,908,3619,773,1018,4637
3,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-09,1031,3810,1017,1030,4840
4,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-10,1086,4174,1103,1247,5421


In [19]:
first_returns = card_visits[card_visits["visit_type"] == "first_return"].copy()

if len(first_returns) > 0:
    intervals = [0, 30, 60, 90, 180, 365]
    labels = ["30", "60", "90", "180", "365"]

    first_returns["conv"] = pd.cut(first_returns["conversion_days"],
       bins=intervals,
       labels=labels,
       include_lowest=True)
    
    # рахуємо, скільки клієнтів у кожному
    conv_counts = (
        first_returns
        .groupby(["venue_chain_id", "venue_id", "month", "conv"])
        .size()
        .unstack("conv", fill_value=0)
        .reset_index()
    )

    # переіменовуємо колонки
    conv_counts = conv_counts.rename(columns={
        "30": "conv_30_days",
        "60": "conv_60_days",
        "90": "conv_90_days",
        "180": "conv_180_days",
        "365": "conv_365_days",
    })

else:
    # якщо не було жодного first_return → створюємо пусті колонки з нулями
    conv_counts = monthly_venue[["venue_chain_id", "venue_id", "month"]].copy()
    for col in ["conv_30_days", "conv_60_days", "conv_90_days", "conv_180_days", "conv_365_days"]:
        conv_counts[col] = 0

/var/folders/wt/vvgcqc6j2kzfmfj6fwh8lzk40000gn/T/ipykernel_26138/3025015383.py:14: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  .groupby(["venue_chain_id", "venue_id", "month", "conv"])


In [21]:
monthly_venue = (
    pd.merge(
        monthly_venue,          
        conv_counts,           
        on=["venue_chain_id", "venue_id", "month"],
        how="left"               
    )
    .fillna(0)                  
)

# рахую кількість повертаючих клієнтів (first_return + repeated_return)
monthly_venue["returning_customers"] = (
    monthly_venue["first_return_customers"] +
    monthly_venue["repeated_return_customers"]
)

# перетворюю місяць у період для коректного сортування та rolling
monthly_venue["month_period"] = monthly_venue["month"].apply(lambda x: pd.Period(x, freq="M"))
monthly_venue = monthly_venue.sort_values(["venue_chain_id", "venue_id", "month_period"])

# обчислення виконуються по кожному venue
group_cols = ["venue_chain_id", "venue_id"]

# сумарна кількість нових клієнтів за останні 12 місяців
monthly_venue["rolling_new_12m"] = (
    monthly_venue.groupby(group_cols)["new_customers"]
    .rolling(12, min_periods=1).sum()
    .reset_index(level=group_cols, drop=True)
)

# сумарна кількість повертаючих клієнтів за 12 місяців
monthly_venue["rolling_returning_12m"] = (
    monthly_venue.groupby(group_cols)["returning_customers"]
    .rolling(12, min_periods=1).sum()
    .reset_index(level=group_cols, drop=True)
)

# середня кількість нових та повертаючих клієнтів за місяць (на основі 12M rolling)
monthly_venue["avg_new_customers_12m"] = monthly_venue["rolling_new_12m"] / 12
monthly_venue["avg_returning_customers_12m"] = monthly_venue["rolling_returning_12m"] / 12

# переглядаю результат
monthly_venue.head()

,venue_chain_id,venue_id,month,first_return_customers,new_customers_card,repeated_return_customers,new_customers_cash,new_customers,conv_30_days,conv_60_days,conv_90_days,conv_180_days,conv_365_days,returning_customers,month_period,rolling_new_12m,rolling_returning_12m,avg_new_customers_12m,avg_returning_customers_12m
0,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-06,611,3396,225,747,4143,611,0,0,0,0,836,2021-06,4143.0,836.0,345.250000,69.666667
1,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-07,1014,4472,641,1050,5522,935,79,0,0,0,1655,2021-07,9665.0,2491.0,805.416667,207.583333
2,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-08,908,3619,773,1018,4637,719,146,43,0,0,1681,2021-08,14302.0,4172.0,1191.833333,347.666667
3,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-09,1031,3810,1017,1030,4840,780,111,109,31,0,2048,2021-09,19142.0,6220.0,1595.166667,518.333333
4,14eaeec5-2fd0-621a-49aa-4a8090cabcf7,1,2021-10,1086,4174,1103,1247,5421,745,136,86,119,0,2189,2021-10,24563.0,8409.0,2046.916667,700.750000


In [25]:
final_cols = [
    "venue_chain_id", "venue_id", "month",
    "new_customers",
    "first_return_customers",
    "repeated_return_customers",
    "returning_customers",
    "conv_30_days", "conv_60_days", "conv_90_days", "conv_180_days", "conv_365_days",
    "avg_new_customers_12m", "avg_returning_customers_12m",
]

monthly_venue_final = monthly_venue[final_cols].copy()
monthly_venue_final.head()final_cols = [
    "venue_chain_id", "venue_id", "month",
    "new_customers",
    "first_return_customers",
    "repeated_return_customers",
    "returning_customers",
    "conv_30_days", "conv_60_days", "conv_90_days", "conv_180_days", "conv_365_days",
    "avg_new_customers_12m", "avg_returning_customers_12m",
]

monthly_venue_final = monthly_venue[final_cols].copy()
monthly_venue_final.head()

output_path = os.path.join(DATA_DIR, "monthly_stats_by_venue.csv")
monthly_venue_final.to_csv(output_path, index=False)
output_path

SyntaxError: invalid syntax (3427784367.py, line 12)